This scripts is revised from
  https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

where this post implemented a classification tree, here I revised it as regression tree. Compared with the implementation did by myself, this script's design of "node" is better, also the author has a better implementation of recursive programming. And actually this is an example of Depth-First-Search (DFS).

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
def squared_loss(y, left, right):
    """
    calculate squared loss for a split
    """
    return np.sum((y[left] - np.mean(y[left])) ** 2) + np.sum((y[right] - np.mean(y[right])) ** 2)

def split_feature(X, y, feature_idx, split):
    """
    split a data set on an attribute and an attribute value
    """
    left, right = list(), list()
    for row_idx in range(X.shape[0]):
        if X[row_idx, feature_idx] < split:
            left.append(row_idx)
        else:
            right.append(row_idx)
    
    return left, right

def get_optim_split(X, y):
    """
    search for the best split
    """
    # child_idx denote the index of observations in left child and right child of the node
    idx, split, loss, child_idx = np.inf, np.inf, np.inf, [np.inf, np.inf]
    for feature_idx in range(X.shape[1]):
        for row_idx in range(X.shape[0]):
            left, right = split_feature(X, y, feature_idx, X[row_idx, feature_idx])
            cost = squared_loss(y, left, right)
            if cost < loss:
                idx, split, loss, child_idx = feature_idx, X[row_idx, feature_idx], cost, [left, right]
    
    return {'feature': idx, 'split': split, 'loss': loss, 'child_idx': child_idx}

In [3]:
def to_leaf(y):
    return np.mean(y)

def split(X, y, node, max_depth, min_split_sample, depth):
    left, right = node['child_idx']
    del node['child_idx']
    
    # check for no split
    if not left or not right:
        # node['left'] and node['right'] are the mean of left child and right child of a node
        node['left'] = node['right'] = to_leaf(y)
        return
    
    # check for maximum allowed depth
    if depth >= max_depth:
        node['left'], node['right'] = to_leaf(y[left]), to_leaf(y[right])
        return

    # split left
    if len(left) <= min_split_sample:
        node['left'] = to_leaf(y[left])
    else:
        node['left'] = get_optim_split(X[left, :], y[left])
        split(X[left, :], y[left], node['left'], max_depth, min_split_sample, depth + 1)
    
    # split right
    if len(right) <= min_split_sample:
        node['right'] = to_leaf(y[right])
    else:
        node['right'] = get_optim_split(X[right, :], y[right])
        split(X[right, :], y[right], node['right'], max_depth, min_split_sample, depth + 1)

def build_tree(X, y, max_depth, min_split_sample):
    """
    build the decision from root
    """
    root = get_optim_split(X, y)
    split(X, y, root, max_depth, min_split_sample, 1)
    return root

def print_tree(node, depth = 0):
    """
    visualize the tree
    """
    # check if the node is a dictionary, if it is then it is not a leaf
    if isinstance(node, dict):
        print('{}[X{} < {}]'.format((depth * '-'), node['feature'], node['split']))
        print_tree(node['left'], depth + 1)
        print_tree(node['right'], depth + 1)
    else:
        print('{}[{}]'.format(depth * '-', node))

def predict(node, row_idx):
    if row_idx[node['feature']] < node['split']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row_idx)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row_idx)
        else:
            return node['right']

In [4]:
os.chdir('C:\Users\Bangda\Desktop')
df = pd.read_csv('df.csv')
X_train = df.iloc[:, 1:].values
y_train = df.iloc[:, 0].values

In [5]:
tree = build_tree(X_train, y_train, 5, 5)
print_tree(tree)

C:\Users\Bangda\Anaconda2\lib\site-packages\numpy\core\fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Bangda\Anaconda2\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[X0 < 6.812]
-[X1 < 1.1742]
--[50.0]
--[X1 < 2.0869]
---[X0 < 4.368]
----[25.3]
----[X1 < 1.3216]
-----[20.85]
-----[13.278]
---[X0 < 6.549]
----[X0 < 6.108]
-----[19.7587301587]
-----[21.6762711864]
----[X1 < 2.6775]
-----[14.5]
-----[26.6814814815]
-[X0 < 7.454]
--[X1 < 1.4655]
---[10.4]
---[X1 < 3.4952]
----[X1 < 2.1675]
-----[33.84]
-----[25.8]
----[X0 < 6.951]
-----[29.25]
-----[33.9583333333]
--[X1 < 2.872]
---[50.0]
---[X0 < 8.069]
----[X0 < 7.831]
-----[44.325]
-----[49.5]
----[43.375]


In [6]:
root = build_tree(X_train, y_train, 5, 5)
predictions = []
for row_idx in range(X_train.shape[0]):
    prediction = predict(root, X_train[row_idx, :])
    predictions.append(prediction)

In [7]:
print('RMSE: {}'.format(np.sqrt(np.mean((predictions - y_train) ** 2))))

RMSE: 3.5117882709
